In [ ]:
import pandas as pd
from openai import OpenAI
import json
import time

# ============================================================
# 01. Cargar configuración desde JSON
# ============================================================
with open("ConfigClasificadorAgro.json", "r", encoding="utf-8") as f:
    config = json.load(f)

API_KEY       = config["OPENAI_API_KEY"]
INPUT_XLSX    = config["INPUT_XLSX"]
OUTPUT_XLSX   = config["OUTPUT_XLSX"]
SHEET_NAME    = config["SHEET_NAME"]
use_cols      = config["USE_COLS"]
SLEEP_SECONDS = config["SLEEP_SECONDS"]
MODEL         = config["MODEL"]

# Inicializar cliente con la API Key del JSON
client = OpenAI(api_key=API_KEY)

# ============================================================
# PROMPTS ADAPTADOS A MAQUINARIA AGRÍCOLA
# ============================================================
PROMPT_DIRECT = """
Eres un clasificador muy estricto de maquinaria agrícola.
Analiza únicamente esta descripción de producto:

"{descripcion}"

Reglas:
1. Devuelve SOLO el JSON plano, sin comentarios ni bloques de código.
2. Para la clave "Marca":
   - Devuelve exactamente lo que aparezca en el texto, en MAYÚSCULA.
   - Si hay varias marcas explícitas, sepáralas con coma.
   - Si no hay marca, devuelve "DESCONOCIDA".
   - No inventes datos irrelevantes (ej: si el texto menciona un motor Perkins en un tractor JOHN DEERE, la marca principal es JOHN DEERE, no PERKINS).
3. Para la clave "Condicion":
   - Devuelve "USADO" si aparecen palabras como "usado", "segunda","reacondicionado","remanufacturado","refurbished","utilizado","segunda mano" o un año que no sea el vigente.
   - Si no aparecen, devuelve "NUEVO".

Responde SOLO en JSON válido con este formato:
{{"Marca": "...", "Condicion": "..."}}
"""

PROMPT_INFER = """
Tengo esta descripción de maquinaria agrícola: "{descripcion}".

Reglas:
1. Si no hay marca explícita, usa seriales, referencias o modelos
   para INFERIR la marca más probable.
   Ejemplos:
   - "S790": JOHN DEERE
   - "AFS CONNECT MAGNUM": CASE IH
   - "CR8.90": NEW HOLLAND
   - "MF 7720": MASSEY FERGUSON
   - "LEXION 770": CLAAS
   - "M7-171": KUBOTA
   - "FENDT 1050 VARIO": FENDT
   - "VALTRA T234": VALTRA
   - "QUOTA 4WD 180": QUOTA
2. Devuelve en MAYÚSCULA.
3. Si hay varias posibles, sepáralas con coma.
4. Si no se puede inferir, responde "DESCONOCIDA".
5. Devuelve SOLO JSON plano, sin bloques de código.

Formato:
{{"Marca_probable": "..."}}
"""

# ============================================================
# FUNCIÓN CLASIFICADORA
# ============================================================
def clasificar_producto(descripcion: str):
    # --- Paso 1: extracción directa ---
    resp = client.responses.create(
        model=MODEL,
        input=PROMPT_DIRECT.format(descripcion=descripcion)
    )
    text = resp.output[0].content[0].text.strip()
    try:
        result = json.loads(text)
        marca = result.get("Marca", "DESCONOCIDA").upper()
        condicion = result.get("Condicion", "NUEVO").upper()
    except:
        print("Error parseando JSON directo. Respuesta:", text)
        marca, condicion = "DESCONOCIDA", "NUEVO"

    # --- Paso 2: inferencia si no hay marca ---
    marca_probable = marca
    if marca == "DESCONOCIDA":
        resp2 = client.responses.create(
            model=MODEL,
            input=PROMPT_INFER.format(descripcion=descripcion)
        )
        text2 = resp2.output[0].content[0].text.strip()
        try:
            result2 = json.loads(text2)
            marca_probable = result2.get("Marca_probable", "DESCONOCIDA").upper()
        except:
            print("Error parseando JSON inferencia. Respuesta:", text2)
            marca_probable = "DESCONOCIDA"

    return marca, marca_probable, condicion

# ============================================================
# PROCESAR TODO EL DF
# ============================================================
marcas, marcas_probables, condiciones = [], [], []

df = pd.read_excel(INPUT_XLSX, sheet_name=SHEET_NAME, usecols=use_cols, engine="openpyxl")
print("Columnas cargadas:", df.columns.tolist())

for i, desc in enumerate(df["Descripción de la mercancía"]):
    marca, marca_prob, condicion = clasificar_producto(desc)
    marcas.append(marca)
    marcas_probables.append(marca_prob)
    condiciones.append(condicion)

    #  Mostrar en consola resultado de la fila
    print(f"\nFila {i+1}:")
    print(f" Descripción: {desc}")
    print(f" Marca: {marca}")
    print(f" Marca probable: {marca_prob}")
    print(f" Condición: {condicion}")
    print("-" * 60)

    if (i+1) % 20 == 0:
        print(f" Procesadas {i+1}/{len(df)} filas...\n")
    time.sleep(SLEEP_SECONDS)

df["Marca"] = marcas
df["Marca_probable"] = marcas_probables
df["Condicion"] = condiciones

# ============================================================
# GUARDAR RESULTADOS
# ============================================================
df.to_excel(OUTPUT_XLSX, index=False)
print("\n Archivo generado:", OUTPUT_XLSX)
